# Generate Test Data

In [ ]:
import xarray as xr
import numpy as np
import dask
import intake
from getpass import getuser
from pathlib import Path

import marEx
import marEx.helper as hpc

In [ ]:
# Lustre Scratch Directory
scratch_dir = Path('/scratch') / getuser()[0] / getuser()
save_dir = Path('/home') / getuser()[0] / getuser() / 'opt' / 'marEx' / 'tests' / 'data'

In [ ]:
# Start Dask Cluster
client = hpc.start_local_cluster(n_workers=32, threads_per_worker=1,
                                 scratch_dir = scratch_dir / 'clients')  # Specify temporary scratch directory for dask to use

2025-06-27 11:39:31,250 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:34833' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('any-aggregate-e47cb6623be3b61b52b0aaf0cbe1057e', 0, 0)} (stimulus_id='handle-worker-cleanup-1751017171.250846')
2025-06-27 11:39:31,257 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:33261' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('rechunk-merge-4c300c227b6f6f73fd5c6bef03f284f7', 4, 0, 0), ('rechunk-merge-4c300c227b6f6f73fd5c6bef03f284f7', 9, 0, 0), ('rechunk-merge-4c300c227b6f6f73fd5c6bef03f284f7', 8, 0, 0), ('rechunk-merge-4c300c227b6f6f73fd5c6bef03f284f7', 1, 0, 0), ('rechunk-merge-4c300c227b6f6f73fd5c6bef03f284f7', 6, 0, 0), ('rechunk-merge-4c300c227b6f6f73fd5c6bef03f284f7', 0, 0, 0), ('rechunk-merge-4c300c227b6f6f73fd5c6bef03f284f7', 2, 0, 0), ('rechunk-merge-4c300c227b6f6f73fd5c6bef03f284f7', 5, 0, 0), ('rechunk-merge-4c

# Raw SST Data

In [ ]:
cat = intake.open_catalog("https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/eerie.yaml")

In [ ]:
# Import 40 years of OSTIA Observations
dat_regrid = cat['dkrz.disk.observations.OSTIA.OSTIA.daily']
sst_regrid = dat_regrid(chunks={}).to_dask().sst.astype(np.float32).coarsen(lat=5,lon=5).mean().rename('to')
sst_regrid

In [ ]:
# Import 40 years of Daily ICON data (ref. EERIE project)
dat_native = cat['dkrz.disk.model-output.icon-esm-er.eerie-control-1950.v20240618']['ocean']['native']
sst_native = dat_native['2d_daily_mean'](chunks={}).to_dask().to.isel(depth=0).drop_vars({'depth','cell_sea_land_mask'}).chunk({'time':32})
sst_native

In [ ]:
# Define subset
time_slice_ostia = slice('1982-01-01', '2022-01-01')
time_slice_icon = slice('1991-01-01', '2031-01-01')
regrid_slice_ostia = dict(lat=slice(35, 40), lon=slice(-40, -30))
native_slice_icon = slice(0,1000)

In [ ]:
sst_regrid_slice = sst_regrid.sel(time=time_slice_ostia, **regrid_slice_ostia).chunk({'time':30, 'lat':-1,'lon':-1})
sst_regrid_slice

In [ ]:
sst_native_slice = sst_native.sel(time=time_slice_icon).isel(ncells=native_slice_icon).drop_vars({'lat','lon'})
sst_native_slice

In [ ]:
# Save to Zarr
sst_regrid_slice.to_zarr(save_dir / 'sst_gridded.zarr', mode='w')
sst_native_slice.to_zarr(save_dir / 'sst_unstructured.zarr', mode='w')

# Pre-processed Data

In [ ]:
time_slice = slice('2010-01-01', '2010-02-01')
extremes_ds = xr.open_zarr(scratch_dir / 'mhws' / 'extremes_binary_gridded_shifting_hobday.zarr', chunks={}).sel(time=time_slice).drop_vars({'thresholds','dat_anomaly','dayofyear'})
extremes_ds

In [ ]:
# Coarsen extremes_ds (lat & lon)
extremes_ds_coarsen = extremes_ds.coarsen(lat=4, lon=4, boundary='trim').any().chunk({'time': 2, 'lat':-1, 'lon':-1}).persist()
extremes_ds_coarsen

In [ ]:
# Clear encoding so we can write to Zarr
extremes_ds_coarsen.encoding = {}
for var in extremes_ds_coarsen.data_vars:
	extremes_ds_coarsen[var].encoding = {}

In [ ]:
extremes_ds_coarsen.to_zarr(save_dir / 'extremes_gridded.zarr', mode='w')